In [3]:
import numpy as np

# Inicialización de Pesos

In [1]:
def initialize_weights(layer_sizes):
    """
    Inicializa los pesos y biases para una red de múltiples capas.

    Parameters:
        layer_sizes : list of int
            Lista donde cada elemento es el número de neuronas de cada capa (incluyendo capa de entrada y de salida).

    Returns:
        weights : list of np.array
            Lista de matrices de pesos para cada capa.
        biases : list of np.array
            Lista de vectores de bias para cada capa.
    """
    weights = []
    biases = []

    # Inicializamos los pesos y biases entre cada capa de la red
    for i in range(1, len(layer_sizes)):
        W = np.random.randn(layer_sizes[i - 1], layer_sizes[i]) * 0.01
        b = np.zeros((1, layer_sizes[i]))
        weights.append(W)
        biases.append(b)

    return weights, biases

In [12]:
layer_sizes = [4, 10, 8, 3] 
weights, biases = initialize_weights(layer_sizes)
print(weights[2])

[[-0.00146973 -0.00410816  0.01000724]
 [-0.00521095  0.00777858 -0.01853591]
 [ 0.01106299  0.01062594 -0.00285421]
 [-0.00242291 -0.01188509 -0.01484589]
 [ 0.02448822  0.00580446 -0.00066426]
 [-0.01063403 -0.0023246  -0.01287589]
 [-0.00833861  0.0097587   0.01816757]
 [-0.02004078  0.0012141  -0.00641758]]


# Funciones de Activación

## ReLu

In [22]:
# Función de activación ReLU
def relu(x):
    return np.maximum(0, x)

# Derivada de ReLU para backpropagation
def relu_derivative(x):
    return np.where(x > 0, 1, 0)

## Sigmoide

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

## Softmax

In [23]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Evita overflow numérico
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [14]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [15]:
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# Forward Propagation

In [20]:
def forward_propagation(X, weights, biases): # , activations
    """
    Realiza la propagación hacia adelante en una red de múltiples capas.

    Parameters:
        X : np.array
            Entrada de la red.
        weights : list of np.array
            Lista de matrices de pesos para cada capa.
        biases : list of np.array
            Lista de vectores de bias para cada capa.
        activations : list of functions
            Funciones de activación para cada capa.

    Returns:
        activations_list : list of np.array
            Lista de activaciones para cada capa.
        zs : list of np.array
            Lista de valores z (pre-activación) para cada capa.
    """
    activations_list = [X]
    zs = []
    activation = X

    # Propagación hacia adelante para todas las capas menos la última # Los z deberían ser los a del profe y viceversa
    for i in range(len(weights) - 1):
        z = np.dot(activation, weights[i]) + biases[i]
        zs.append(z)
        activation = relu(z)
        activations_list.append(activation)

    # Última capa con softmax
    z = np.dot(activation, weights[-1]) + biases[-1]
    zs.append(z)
    activation = softmax(z)
    activations_list.append(activation)

    return activations_list, zs

In [ ]:
activations_list, zs = forward_propagation(X, weights, biases)
print(zs)

# Función de Coste

In [ ]:
def compute_cost(Y_pred, Y_true):
    """
    Calcula el costo usando la entropía cruzada categórica.

    Parameters:
        Y_pred : np.array
            Salidas predichas de la red (después de softmax).
        Y_true : np.array
            Etiquetas en formato one-hot.

    Returns:
        cost : float
            El valor del costo (entropía cruzada categórica).
    """
    m = Y_true.shape[0]
    cost = -np.sum(Y_true * np.log(Y_pred)) / m
    return cost

# Backward Propagation

In [ ]:
def backward_propagation(Y, weights, biases, activations_list, zs, activation_derivatives):
    """
    Realiza la retropropagación en una red de múltiples capas para calcular los gradientes.

    Parameters:
        Y : np.array
            Etiquetas en formato one-hot.
        weights : list of np.array
            Lista de matrices de pesos para cada capa.
        biases : list of np.array
            Lista de vectores de bias para cada capa.
        activations_list : list of np.array
            Lista de activaciones para cada capa.
        zs : list of np.array
            Lista de valores z (pre-activación) para cada capa.
        activation_derivatives : list of functions
            Derivadas de las funciones de activación para cada capa.

    Returns:
        nabla_b : list of np.array
            Gradientes de los biases para cada capa.
        nabla_w : list of np.array
            Gradientes de los pesos para cada capa.
    """
    num_layers = len(weights)
    nabla_b = [np.zeros(b.shape) for b in biases]
    nabla_w = [np.zeros(w.shape) for w in weights]

    # Cálculo del error en la capa de salida
    delta = activations_list[-1] - Y        # Es lo mismo que usar entropía cruzada
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(activations_list[-2].T, delta) # Creo que hay que quitarlo / Y.shape[0]

    # Retropropagación a través de las capas ocultas
    for l in range(2, num_layers + 1):
        z = zs[-l]
        sp = activation_derivatives[-l](z)
        delta = np.dot(delta, weights[-l + 1].T) * sp
        nabla_b[-l] = delta
        nabla_w[-l] = np.dot(activations_list[-l - 1].T, delta) # Esto también / Y.shape[0]

    return nabla_b, nabla_w